In [1]:
%cd ../..

/home/work/AnacondaProjects/combinators


In [2]:
import logging

import numpy as np
import probtorch
import torch

from examples.fep_control import fep_control
from combinators.model import active
from combinators.model import compose, foldable
from combinators.inference import importance
from combinators import utils

In [3]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [4]:
target = fep_control.MountainCarStep(state_dim=2, action_dim=1, observation_dim=2, batch_shape=(1,), discrete_actions=False, trainable=False)
proposal = fep_control.RecognitionStep(state_dim=2, observation_dim=2, batch_shape=(1,), discrete_actions=False, name='MountainCarStep', trainable=True)
agent = importance.propose(target, proposal)

In [5]:
theta, graph, log_weight = active.active_inference_test(agent, 'MountainCarContinuous-v0', online_inference=False, iterations=1000)

In [6]:
theta, graph, log_weight = active.active_inference(agent, 'MountainCarContinuous-v0', lr=1e-3, episodes=10, episode_length=1000, dream=False)

04/25/2019 18:24:15 ELBO=-1.91722168e+04 at episode 0 of length 1000
04/25/2019 18:24:25 ELBO=-1.71693884e+03 at episode 1 of length 67
04/25/2019 18:24:37 ELBO=-8.66627539e+03 at episode 2 of length 1000
04/25/2019 18:24:47 ELBO=-4.38470508e+03 at episode 3 of length 479
04/25/2019 18:24:58 ELBO=-5.43808301e+03 at episode 4 of length 711
04/25/2019 18:25:11 ELBO=-1.59630205e+04 at episode 5 of length 1000
04/25/2019 18:25:21 ELBO=-3.00003320e+03 at episode 6 of length 118
04/25/2019 18:25:30 ELBO=-2.79682935e+03 at episode 7 of length 81
04/25/2019 18:25:39 ELBO=-1.75672925e+03 at episode 8 of length 197
04/25/2019 18:25:50 ELBO=-5.03098145e+03 at episode 9 of length 618


In [7]:
log_weight

tensor([-5030.9814], grad_fn=<AddBackward0>)

In [13]:
theta, graph, log_weight = active.active_inference_test(agent, 'MountainCarContinuous-v0', online_inference=False, iterations=1000)

In [9]:
theta

(tensor([[ 0.0118, -0.4780]], grad_fn=<AddBackward0>),
 tensor([[-0.9226]], grad_fn=<ExpandBackward>))

In [10]:
log_weight

tensor([-12989.1758], grad_fn=<AddBackward0>)

In [11]:
torch.save(agent, 'examples/fep_control/fep_mountain_car_agent.dat')